In [162]:
from tensorflow.python.framework import graph_util
import tensorflow as tf
import numpy as np
import os
import time
from keras.datasets import cifar10
pb_file_path = os.getcwd() + '/'

batch_size=128
max_steps=3000
nb = -1

In [163]:
(trainX, trainY), (testX, testY) = cifar10.load_data()
def nextBatch():
    global nb
    nb = nb + 1
    if nb == 390:
        nb = 0
    return trainX[nb*128:(nb+1)*128], np.reshape(trainY[nb*128:(nb+1)*128], (128,))

In [164]:
###定义权重的函数###
#使用截断的正态分布初始化权重变量
#如果wl不为0，则对对于权重添加一个L2范数的正则化，且命名为'weight_loss'，
#然后将这个正则化部分添加到名为'losses'的collection中，方便后续调用；
def variable_with_weight_loss(shape,stddev,wl,name):
    var=tf.Variable(tf.truncated_normal(shape,stddev=stddev), name=name)
    if wl is not None:
        weight_loss=tf.multiply(tf.nn.l2_loss(var),wl,name='weight_loss')
        tf.add_to_collection('losses',weight_loss)
    return var

###定义损失函数，将L2也加入其中###
def loss(logits,labels):
    labels = tf.cast(labels, tf.int64)
    #使用softmax分类器，然后结合交叉熵作为损失函数
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = labels, name = 'cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name = 'cross_entropy')
    #将平均后的损失函数也添加进入‘losses’
    tf.add_to_collection('losses', cross_entropy_mean)
    #tf.get_collection('losses')，将之前添加到列表losses中的内容输出成一个数组
    #使用tf.add_n，将列表losses中的所有项求和；
    return tf.add_n(tf.get_collection('losses'), name = 'total_loss')

In [165]:
graph = tf.Graph()
with graph.as_default():

    #创建输入图像和标签所需要的占位符（placeholder），batch_size是一次输入的图像数目，32*32是裁图像大小，3是图像的通道数（输入为彩色图）
    image_holder=tf.placeholder(tf.float32,[batch_size,32,32,3], name='image')
    label_holder=tf.placeholder(tf.int32,[batch_size], name='label')

    ###创建第一个卷积层####这一层有64个节点
    #调用variable_with_weight_loss函数创建第一个卷积核，大小为5*5，通道数为3，标准差为0.05，不设置权重损失；
    weight1=variable_with_weight_loss(shape=[5,5,3,64],stddev=5e-2,wl=0.0, name='weight1')
    kernel1=tf.nn.conv2d(image_holder,weight1,[1,1,1,1],padding='SAME')
    bias1=tf.Variable(tf.constant(0.0,shape=[64]), name='bias1')
    conv1=tf.nn.relu(tf.nn.bias_add(kernel1,bias1))
    pool1=tf.nn.max_pool(conv1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
    #对上面处理后的记过进行LRN处理。
    norm1=tf.nn.lrn(pool1,4,bias=1.0,alpha=0.001/9.0,beta=0.75)
    
    
    ###创建第二个卷积层###这一层有64个节点
    weight2=variable_with_weight_loss(shape=[5,5,64,64],stddev=5e-2,wl=0.0, name='weight2')
    kernel2=tf.nn.conv2d(norm1,weight2,[1,1,1,1],padding='SAME')
    bias2=tf.Variable(tf.constant(0.1,shape=[64]), name='bias2')
    conv2=tf.nn.relu(tf.nn.bias_add(kernel2,bias2))
    ###先进行LRN处理再做最大值池化
    norm2=tf.nn.lrn(conv2,4,bias=1.0,alpha=0.001/9.0,beta=0.75)
    pool2=tf.nn.max_pool(norm2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')

    ###创建全连接层### 这一层有384个节点
    #首先将图像展开，这里的维度第一个是多少张图，第二个是根据图像大小确定的；然后使用get_shape()获取第二个维度，也就是图像大小
    reshape=tf.reshape(pool2,[batch_size,-1])
    dim=reshape.get_shape()[1].value

    ###第一个全连接层，有384个节点###
    weight3=variable_with_weight_loss(shape=[dim,384],stddev=0.04,wl=0.004, name='weight3')
    bias3=tf.Variable(tf.constant(0.1,shape=[384]), name='bias3')
    local3=tf.nn.relu(tf.matmul(reshape,weight3)+bias3)
     
    ###第二个全连接层，有192个节点
    weight4=variable_with_weight_loss(shape=[384,192],stddev=0.04,wl=0.004, name='weight4')
    bias4=tf.Variable(tf.constant(0.1,shape=[192]), name='bias4')
    local4=tf.nn.relu(tf.matmul(local3,weight4)+bias4)

    ###第三个全连接层，有10个节点
    weight5=variable_with_weight_loss(shape=[192,10],stddev=1/192.0,wl=0.0, name='weight5')
    bias5=tf.Variable(tf.constant(0.0,shape=[10]), name='bias5')
    logits=tf.add(tf.matmul(local4,weight5),bias5, name="logits")

    
    # Loss
    loss = loss(logits, label_holder)
    
    #使用Adam Optimizer优化模型，学习率为0.001；
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

    ##tf.nn.in_top_k()该函数首先将logits的输出排序，然后根据第三个参数选取排序的前N个，这里是1，就是选取第一个与label_holder进行对比；
    ##比如logits最大的是第五个，而label_holder也是5；那么输出为True；
    top_k_op = tf.nn.in_top_k(logits, label_holder, 1, name="top_k_op")


In [166]:
with tf.Session(graph=graph) as sess:
    
    sess.run(tf.global_variables_initializer())
    
    for step in range(max_steps):
        start_time = time.time()
        image_batch, label_batch = nextBatch()
        _, loss_value = sess.run([train_op, loss], feed_dict = {image_holder: image_batch, label_holder: label_batch})
        duration = time.time() - start_time
        if step % 10 == 0:
            examples_per_sec = batch_size / duration
            sec_per_batch = float(duration)

            format_str = ('step %d, loss = %.2f(%.1f examples/sec; %.3f sec/batch)')
            print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))
            
            predictions = sess.run([top_k_op], feed_dict = {image_holder: testX[0:128], label_holder: np.reshape(testY[0:128], (128,))})
            precision = np.sum(predictions) / float(predictions[0].shape[0])
            print('precision @ %d = %.3f' % (step, precision))
            
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['logits', 'total_loss', 'top_k_op/top_k_op'])
    
    with tf.gfile.FastGFile(pb_file_path+'complex_cnn_cifar10_model.pb', mode='wb') as f:
        f.write(constant_graph.SerializeToString())

step 0, loss = 7.05(41.4 examples/sec; 3.093 sec/batch)
precision @ 0 = 0.078
step 10, loss = 5.61(47.3 examples/sec; 2.708 sec/batch)
precision @ 10 = 0.078
step 20, loss = 4.94(47.2 examples/sec; 2.712 sec/batch)
precision @ 20 = 0.164
step 30, loss = 4.42(45.3 examples/sec; 2.826 sec/batch)
precision @ 30 = 0.125
step 40, loss = 3.94(43.1 examples/sec; 2.967 sec/batch)
precision @ 40 = 0.211
step 50, loss = 3.63(46.1 examples/sec; 2.777 sec/batch)
precision @ 50 = 0.281
step 60, loss = 3.34(47.5 examples/sec; 2.696 sec/batch)
precision @ 60 = 0.328
step 70, loss = 3.28(47.2 examples/sec; 2.713 sec/batch)
precision @ 70 = 0.297
step 80, loss = 3.04(45.4 examples/sec; 2.817 sec/batch)
precision @ 80 = 0.359
step 90, loss = 2.72(46.5 examples/sec; 2.750 sec/batch)
precision @ 90 = 0.383
step 100, loss = 2.59(47.3 examples/sec; 2.709 sec/batch)
precision @ 100 = 0.367
step 110, loss = 2.64(45.8 examples/sec; 2.795 sec/batch)
precision @ 110 = 0.414
step 120, loss = 2.56(48.1 examples/se

precision @ 1000 = 0.664
step 1010, loss = 1.20(47.9 examples/sec; 2.671 sec/batch)
precision @ 1010 = 0.711
step 1020, loss = 1.25(48.3 examples/sec; 2.649 sec/batch)
precision @ 1020 = 0.641
step 1030, loss = 1.21(48.0 examples/sec; 2.667 sec/batch)
precision @ 1030 = 0.719
step 1040, loss = 1.07(47.4 examples/sec; 2.700 sec/batch)
precision @ 1040 = 0.656
step 1050, loss = 1.23(47.5 examples/sec; 2.694 sec/batch)
precision @ 1050 = 0.695
step 1060, loss = 1.34(47.8 examples/sec; 2.675 sec/batch)
precision @ 1060 = 0.633
step 1070, loss = 1.26(48.1 examples/sec; 2.660 sec/batch)
precision @ 1070 = 0.680
step 1080, loss = 1.06(47.4 examples/sec; 2.700 sec/batch)
precision @ 1080 = 0.594
step 1090, loss = 1.22(47.8 examples/sec; 2.676 sec/batch)
precision @ 1090 = 0.672
step 1100, loss = 1.24(47.9 examples/sec; 2.670 sec/batch)
precision @ 1100 = 0.617
step 1110, loss = 0.99(47.5 examples/sec; 2.695 sec/batch)
precision @ 1110 = 0.656
step 1120, loss = 1.10(46.8 examples/sec; 2.736 sec

precision @ 1980 = 0.680
step 1990, loss = 1.08(47.8 examples/sec; 2.677 sec/batch)
precision @ 1990 = 0.719
step 2000, loss = 1.02(47.5 examples/sec; 2.694 sec/batch)
precision @ 2000 = 0.703
step 2010, loss = 0.99(47.4 examples/sec; 2.699 sec/batch)
precision @ 2010 = 0.727
step 2020, loss = 1.17(47.5 examples/sec; 2.694 sec/batch)
precision @ 2020 = 0.711
step 2030, loss = 1.02(46.2 examples/sec; 2.771 sec/batch)
precision @ 2030 = 0.703
step 2040, loss = 0.94(46.0 examples/sec; 2.784 sec/batch)
precision @ 2040 = 0.727
step 2050, loss = 0.86(47.9 examples/sec; 2.674 sec/batch)
precision @ 2050 = 0.711
step 2060, loss = 0.99(47.0 examples/sec; 2.726 sec/batch)
precision @ 2060 = 0.703
step 2070, loss = 0.87(47.6 examples/sec; 2.686 sec/batch)
precision @ 2070 = 0.734
step 2080, loss = 1.08(47.3 examples/sec; 2.706 sec/batch)
precision @ 2080 = 0.672
step 2090, loss = 1.00(46.5 examples/sec; 2.753 sec/batch)
precision @ 2090 = 0.734
step 2100, loss = 1.00(47.8 examples/sec; 2.676 sec

precision @ 2960 = 0.719
step 2970, loss = 1.02(46.7 examples/sec; 2.743 sec/batch)
precision @ 2970 = 0.711
step 2980, loss = 0.95(47.3 examples/sec; 2.706 sec/batch)
precision @ 2980 = 0.695
step 2990, loss = 0.95(47.7 examples/sec; 2.683 sec/batch)
precision @ 2990 = 0.703
INFO:tensorflow:Froze 10 variables.


I0902 21:59:20.671936 4483290560 graph_util_impl.py:268] Froze 10 variables.


INFO:tensorflow:Converted 10 variables to const ops.


I0902 21:59:20.692440 4483290560 graph_util_impl.py:301] Converted 10 variables to const ops.


In [167]:
##用含有10000张图片的测试集测试准确率，并且打印出来。    
# num_examples = 10000
# import math
 
# num_iter = int(math.ceil(num_examples / batch_size))
# true_count = 0
# total_sample_count = num_iter * batch_size
# step = 0
 
# while step < num_iter:
#     image_batch, label_batch = nextBatch()
#     predictions = sess.run([top_k_op], feed_dict = {image_holder: image_batch, label_holder: label_batch})
#     true_count += np.sum(predictions)
#     step += 1
 
# precision = true_count / total_sample_count
# print('precision @ 1 = %.3f' % precision)


In [168]:
weight1

<tf.Variable 'weight1:0' shape=(5, 5, 3, 64) dtype=float32_ref>